In [605]:
import pandas as pd
import numpy as np
import time
import requests
import re
from bs4 import BeautifulSoup
import pickle
from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time, os
from fake_useragent import UserAgent
import re
import pickle
import datetime
from selenium.webdriver.common.action_chains import ActionChains

ua = UserAgent()
user_agent = {'User-agent': ua.random}
chromedriver = "/Applications/chromedriver"
os.environ["webdriver.chrome.driver"] = chromedriver

In [631]:
def get_reviews(url_list):
    '''Takes a list of Tripadvisor property URLS, scrapes those property pages for data, and returns all scraped data as a df.

    Arguments: list (of URLs)
    Returns: df
    '''
    review_list_df = pd.DataFrame(columns=("Property name", "Property address", "Lat, long", "Reviewer", "Review date", "Date of stay", "Review title", "Full review", "Review link"))

    '''with open("alaska_hotels_redux.pickle", "rb") as to_read:
        review_list_df = pickle.load(to_read)'''

    for url in tqdm(url_list):
        driver = webdriver.Chrome(chromedriver)
        driver.get(url)
        time.sleep(6)
        
        try:
            property_lat_long = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[5]/div/div/div/div/div/div[2]/span/img').get_attribute("src").split("center=")[1].split("&", 1)[0]
        except NoSuchElementException:
            property_lat_long = np.NaN
        soup = BeautifulSoup(driver.page_source)
        time.sleep(5)
        try:
            property_name = soup.find("h1", class_="_1mTlpMC3", id="HEADING").text
        except AttributeError:
            property_name = soup.find("h1", class_="YeV2SlB6 propertyHeading").text
        try:
            property_address = soup.find("span", class_="_3ErVArsu jke2_wbp").text
        except AttributeError:
            property_address = np.NaN
        try:
            disabled_button = soup.find("div", class_="_16gKMTFp").find(class_=re.compile(".+(disabled).*")).text
        except AttributeError:
            disabled_button = "Next"
        page_number = 1
        reviews = soup.find_all("div", class_="_2wrUUKlw _3hFEdNs8")
        if reviews == []:
            reviewer, review_date, date_of_stay, review_title, full_review, review_link = np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN
            review_details_list = [property_name, property_address, property_lat_long, reviewer, review_date, date_of_stay, review_title, full_review, review_link]
            index = len(review_list_df)
            review_list_df.loc[index] = review_details_list
            with open("alaska_hotels_redux.pickle", "wb") as to_write:
                pickle.dump(review_list_df, to_write)
            print(f'No reviews on page {page_number} (last). Done!')
            continue
        else:
            for review in reviews:
                reviewer, review_date = review.find("div", class_="_2fxQ4TOx").find("span").text.split(" wrote a review ")
                print(reviewer)
                if review_date == "Today":
                    review_date = "November 1"
                elif review_date == "Yesterday":
                    review_date = "October 31"
                else:
                    review_date = datetime.datetime.strptime(review_date.split(" ")[0], '%b').strftime('%B') + ' ' + review_date.split(" ")[1]
                review_link = f'https://www.tripadvisor.com{review.find("div", class_="glasR4aX").a["href"]}'
                review_title = review.find("div", class_="glasR4aX").a.span.text
                full_review = review.find("q", class_="IRsGHoPm").text
                if review.find("span", class_="_355y0nZn"):
                    date_of_stay = review.find("span", class_="_34Xs-BQm").text.split(": ")[1]
                else:
                    date_of_stay = np.NaN
                review_details_list = [property_name, property_address, property_lat_long, reviewer, review_date, date_of_stay, review_title, full_review, review_link]
                index = len(review_list_df)
                review_list_df.loc[index] = review_details_list
            print(f'Page {page_number} done!')

            if disabled_button != "Next":
                try:
                    next_button = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[7]/div/div[1]/div[1]/div/div/div[3]/div[8]/div/a')
                    next_button.click()
                except NoSuchElementException:
                    continue
                time.sleep(3.5)
                time.sleep(3)
                soup = BeautifulSoup(driver.page_source)
                while soup.find("div", id="component_14").find("div", class_="_16gKMTFp").find("div", class_="ui_pagination is-centered").find_all("a")[1].text == "Next":
                    page_number += 1
                    soup = BeautifulSoup(driver.page_source)
                    reviews = soup.find_all("div", class_="_2wrUUKlw _3hFEdNs8")
                    for review in reviews:
                        reviewer, review_date = review.find("div", class_="_2fxQ4TOx").find("span").text.split(" wrote a review ")
                        print(reviewer)
                        if review_date == "Today":
                            review_date = "November 1"
                        elif review_date == "Yesterday":
                            review_date = "October 31"
                        else:
                            review_date = datetime.datetime.strptime(review_date.split(" ")[0], '%b').strftime('%B') + ' ' + review_date.split(" ")[1]
                        review_link = f'https://www.tripadvisor.com{review.find("div", class_="glasR4aX").a["href"]}'
                        review_title = review.find("div", class_="glasR4aX").a.span.text
                        full_review = review.find("q", class_="IRsGHoPm").text
                        if review.find("span", class_="_355y0nZn"):
                            date_of_stay = review.find("span", class_="_34Xs-BQm").text.split(": ")[1]
                        else:
                            date_of_stay = np.NaN
                        review_details_list = [property_name, property_address, property_lat_long, reviewer, review_date, date_of_stay, review_title, full_review, review_link]
                        index = len(review_list_df)
                        review_list_df.loc[index] = review_details_list
                    with open("alaska_hotels_redux.pickle", "wb") as to_write:
                        pickle.dump(review_list_df, to_write)
                    print(f'Page {page_number} done!')
                    next_button = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[7]/div/div[1]/div[1]/div/div/div[3]/div[8]/div/a[2]')
                    next_button.click()
                    time.sleep(4)
                    soup = BeautifulSoup(driver.page_source)
                
                page_number += 1
                time.sleep(3.5)
                reviews = soup.find_all("div", class_="_2wrUUKlw _3hFEdNs8")
                for review in reviews:
                    reviewer, review_date = review.find("div", class_="_2fxQ4TOx").find("span").text.split(" wrote a review ")
                    if review_date == "Today":
                        review_date = "November 1"
                    elif review_date == "Yesterday":
                        review_date = "October 31"
                    else:
                        review_date = datetime.datetime.strptime(review_date.split(" ")[0], '%b').strftime('%B') + ' ' + review_date.split(" ")[1]
                    review_link = f'https://www.tripadvisor.com{review.find("div", class_="glasR4aX").a["href"]}'
                    review_title = review.find("div", class_="glasR4aX").a.span.text
                    full_review = review.find("q", class_="IRsGHoPm").text
                    if review.find("span", class_="_355y0nZn"):
                        date_of_stay = review.find("span", class_="_34Xs-BQm").text.split(": ")[1]
                    else:
                        date_of_stay = np.NaN
                    review_details_list = [property_name, property_address, property_lat_long, reviewer, review_date, date_of_stay, review_title, full_review, review_link]
                    index = len(review_list_df)
                    review_list_df.loc[index] = review_details_list
                with open("alaska_hotels_redux.pickle", "wb") as to_write:
                    pickle.dump(review_list_df, to_write)
                print(f'Page {page_number} (last) done!')

            else:
                reviews = soup.find_all("div", class_="_2wrUUKlw _3hFEdNs8")
                if reviews == []:
                    reviewer, review_date, date_of_stay, review_title, full_review, review_link = np.NaN, np.NaN, np.NaN, np.NaN, np.NaN, np.NaN
                    review_details_list = [property_name, property_address, property_lat_long, reviewer, review_date, date_of_stay, review_title, full_review, review_link]
                    index = len(review_list_df)
                    review_list_df.loc[index] = review_details_list
                    with open("alaska_hotels_redux.pickle", "wb") as to_write:
                        pickle.dump(review_list_df, to_write)
                    print(f'No reviews on page {page_number} (last). Done!')
                else:
                    '''for review in reviews:
                        reviewer, review_date = review.find("div", class_="_2fxQ4TOx").find("span").text.split(" wrote a review ")
                        if review_date == "Today":
                            review_date = "November 1"
                        elif review_date == "Yesterday":
                            review_date = "October 31"
                        else:
                            review_date = datetime.datetime.strptime(review_date.split(" ")[0], '%b').strftime('%B') + ' ' + review_date.split(" ")[1]
                        review_link = f'https://www.tripadvisor.com{review.find("div", class_="glasR4aX").a["href"]}'
                        review_title = review.find("div", class_="glasR4aX").a.span.text
                        full_review = review.find("q", class_="IRsGHoPm").span.text
                        if review.find("span", class_="_355y0nZn"):
                            date_of_stay = review.find("span", class_="_34Xs-BQm").text.split(": ")[1]
                        else:
                            date_of_stay = np.NaN
                        review_details_list = [property_name, property_address, property_lat_long, reviewer, review_date, date_of_stay, review_title, full_review, review_link]
                        index = len(review_list_df)
                        review_list_df.loc[index] = review_details_list
                    with open("alaska_hotels_redux.pickle", "wb") as to_write:
                        pickle.dump(review_list_df, to_write)
                    print(f'Page {page_number} (last) done!')'''
            driver.quit()
            time.sleep(1)

    return review_list_df

In [230]:
get_reviews()

Page 1 done!
Page 2 done!
Page 3 done!
Page 4 done!
Page 5 done!
Page 6 done!
Page 7 done!
Page 8 done!
Page 9 done!
Page 10 done!
Page 11 done!
Page 12 done!
Page 13 done!
Page 14 done!
Page 15 done!
Page 16 done!
Page 17 done!
Page 18 done!
Page 19 done!
Page 20 done!
Page 21 done!
Page 22 done!
Page 23 done!
Page 24 done!
Page 25 done!
Page 26 done!
Page 27 done!
Page 28 done!
Page 29 done!
Page 30 done!
Page 31 done!
Page 32 done!
Page 33 done!
Page 34 done!
Page 35 done!
Page 36 done!
Page 37 done!
Page 38 done!
Page 39 done!
Page 40 done!
Page 41 done!
Page 42 done!
Page 43 done!
Page 44 done!
Page 45 done!
Page 46 done!
Page 47 done!
Page 48 done!
Page 49 done!
Page 50 done!
Page 51 done!
Page 52 done!
Page 53 done!
Page 54 done!
Page 55 done!
Page 56 done!
Page 57 done!
Page 58 done!
Page 59 done!
Page 60 done!
Page 61 done!
Page 62 done!
Page 63 done!
Page 64 done!
Page 65 done!
Page 66 done!
Page 67 done!
Page 68 done!
Page 69 done!
Page 70 done!
Page 71 done!
Page 72 done!
P

,Property name,Property address,"Lat, long",Reviewer,Review date,Date of stay,Review title,Full review,Review link
0,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",sunboundAK,October 25,October 2020,Went down hill,I’ve stayed here several times but the last tw...,https://www.tripadvisor.com/ShowUserReviews-g6...
1,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",WinniePDecatur_GA,August 2020,July 2020,Great experience in Anchorage - amazing staff!,We stayed here at the start of our Alaskan vac...,https://www.tripadvisor.com/ShowUserReviews-g6...
2,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",PDS81,October 4,October 2020,Poor value,I have stayed at this property intermittently ...,https://www.tripadvisor.com/ShowUserReviews-g6...
3,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",Liz,October 3,February 2020,"Fantastic Staff, Unforgetable Experience!",I stayed here for an entire month with my fami...,https://www.tripadvisor.com/ShowUserReviews-g6...
4,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",Tmiller,September 2020,September 2020,Ridiculous prices for the service offered.,Embassy Suites has been my go to place when tr...,https://www.tripadvisor.com/ShowUserReviews-g6...
...,...,...,...,...,...,...,...,...,...
1770,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",uphill2,December 2008,December 2008,Really Wonderful,My wife and I just got back from spending Chri...,https://www.tripadvisor.com/ShowUserReviews-g6...
1771,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",Cobyslady,November 2008,November 2008,Great Stay,"This is a wonderful hotel, I just can't say en...",https://www.tripadvisor.com/ShowUserReviews-g6...
1772,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",TallyAttwater,November 2008,October 2008,Nice comfortable experience - good layover,First time in Anchorage and I would highly rec...,https://www.tripadvisor.com/ShowUserReviews-g6...
1773,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",dave1013,September 2008,NaN,Great new hotel in a great location,Wife and I stayed here for two nights in Septe...,https://www.tripadvisor.com/ShowUserReviews-g6...


In [231]:
with open("alaska_hotels.pickle", "rb") as to_read:
    first_hotel_df = pickle.load(to_read)

In [232]:
first_hotel_df

,Property name,Property address,"Lat, long",Reviewer,Review date,Date of stay,Review title,Full review,Review link
0,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",sunboundAK,October 25,October 2020,Went down hill,I’ve stayed here several times but the last tw...,https://www.tripadvisor.com/ShowUserReviews-g6...
1,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",WinniePDecatur_GA,August 2020,July 2020,Great experience in Anchorage - amazing staff!,We stayed here at the start of our Alaskan vac...,https://www.tripadvisor.com/ShowUserReviews-g6...
2,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",PDS81,October 4,October 2020,Poor value,I have stayed at this property intermittently ...,https://www.tripadvisor.com/ShowUserReviews-g6...
3,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",Liz,October 3,February 2020,"Fantastic Staff, Unforgetable Experience!",I stayed here for an entire month with my fami...,https://www.tripadvisor.com/ShowUserReviews-g6...
4,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",Tmiller,September 2020,September 2020,Ridiculous prices for the service offered.,Embassy Suites has been my go to place when tr...,https://www.tripadvisor.com/ShowUserReviews-g6...
...,...,...,...,...,...,...,...,...,...
1770,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",uphill2,December 2008,December 2008,Really Wonderful,My wife and I just got back from spending Chri...,https://www.tripadvisor.com/ShowUserReviews-g6...
1771,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",Cobyslady,November 2008,November 2008,Great Stay,"This is a wonderful hotel, I just can't say en...",https://www.tripadvisor.com/ShowUserReviews-g6...
1772,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",TallyAttwater,November 2008,October 2008,Nice comfortable experience - good layover,First time in Anchorage and I would highly rec...,https://www.tripadvisor.com/ShowUserReviews-g6...
1773,Embassy Suites by Hilton Anchorage,"600 E Benson Blvd, Anchorage, AK 99503-4123","61.192524,-149.871399",dave1013,September 2008,NaN,Great new hotel in a great location,Wife and I stayed here for two nights in Septe...,https://www.tripadvisor.com/ShowUserReviews-g6...


In [264]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tripadvisor.com/Hotels-g28923-Alaska-Hotels.html")
time.sleep(5)
hotels_toggle = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div/div[1]/div[1]/div[5]/div/div/div[2]/div[2]/div[2]/div[4]/div/label')
hotels_toggle.click()
time.sleep(5)
soup = BeautifulSoup(driver.page_source)
urls = []
for i in range(98):
    soup = BeautifulSoup(driver.page_source)
    listings = soup.find("div", id="taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0").find_all("div", class_="prw_rup prw_meta_hsx_responsive_listing ui_section listItem")
    for listing in listings:
        urls.append(f'https://www.tripadvisor.com{listing.find("div", class_="listing_title").find("a")["href"]}')
    next_button = driver.find_element_by_xpath('//*[@id="taplc_main_pagination_bar_dusty_hotels_resp_0"]/div/div/div/span[2]')
    next_button.click()
soup = BeautifulSoup(driver.page_source)
listings = soup.find("div", id="taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0").find_all("div", class_="prw_rup prw_meta_hsx_responsive_listing ui_section listItem")
for listing in listings:
    urls.append(f'https://www.tripadvisor.com{listing.find("div", class_="listing_title").find("a")["href"]}')
urls


0880-d17384956-Reviews-Hyatt_Place_Anchorage_Midtown-Anchorage_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g60873-d73131-Reviews-Hotel_Seward-Seward_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g60880-d248011-Reviews-Puffin_Inn_of_Anchorage-Anchorage_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g60880-d225230-Reviews-Coast_Inn_at_Lake_Hood-Anchorage_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g31009-d15004443-Reviews-Aspen_Suites_Hotel_Homer-Homer_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g60826-d292982-Reviews-Northern_Sky_Lodge-Fairbanks_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g60880-d11668679-Reviews-Aptel_Studio_Hotel-Anchorage_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g60880-d239349-Reviews-Anchorage_Grand_Hotel-Anchorage_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g31127-d230851-Reviews-Aspen_Hotel_Soldotna-Soldotna_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review

In [241]:
listings = soup.find("div", id="taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0").find_all("div", class_="prw_rup prw_meta_hsx_responsive_listing ui_section listItem")
for listing in listings:
    urls.append(f'https://www.tripadvisor.com{listing.find("div", class_="listing_title").find("a")["href"]}'))

https://www.tripadvisor.com/Hotel_Review-g60873-d8565849-Reviews-Harbor_360_Hotel-Seward_Alaska.html
https://www.tripadvisor.com/Hotel_Review-g60826-d577141-Reviews-Pike_s_Waterfront_Lodge-Fairbanks_Alaska.html
https://www.tripadvisor.com/Hotel_Review-g60880-d72177-Reviews-The_Hotel_Captain_Cook-Anchorage_Alaska.html
https://www.tripadvisor.com/Hotel_Review-g60880-d73014-Reviews-The_Lakefront_Anchorage-Anchorage_Alaska.html
https://www.tripadvisor.com/Hotel_Review-g60880-d73025-Reviews-The_Voyager_Inn-Anchorage_Alaska.html
https://www.tripadvisor.com/Hotel_Review-g60826-d276913-Reviews-Sophie_Station_Suites-Fairbanks_Alaska.html
https://www.tripadvisor.com/Hotel_Review-g60826-d628949-Reviews-La_Quinta_Inn_Suites_by_Wyndham_Fairbanks_Airport-Fairbanks_Alaska.html
https://www.tripadvisor.com/Hotel_Review-g60826-d224159-Reviews-SpringHill_Suites_by_Marriott_Fairbanks-Fairbanks_Alaska.html
https://www.tripadvisor.com/Hotel_Review-g60880-d1097991-Reviews-Embassy_Suites_by_Hilton_Anchorage-A

In [266]:
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as wait

In [280]:
driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tripadvisor.com/Hotels-g28923-oa20-Alaska-Hotels.html#LEAF_GEO_LIST")
soup = BeautifulSoup(driver.page_source)
for item in soup.find("ul", class_="geoList ui_columns is-multiline").find_all("li", class_="ui_column is-12-mobile is-4-tablet is-3-desktop"):
    print(f'https://www.tripadvisor.com{item.a["href"]}')


/Hotels-g2161460-Kodiak_Kodiak_Island_Alaska-Hotels.html
/Hotels-g31131-Sterling_Alaska-Hotels.html
/Hotels-g31091-Petersburg_Alaska-Hotels.html
/Hotels-g13424126-Lake_and_Peninsula_Borough_Alaska-Hotels.html
/Hotels-g31001-Glennallen_Alaska-Hotels.html
/Hotels-g31164-Willow_Alaska-Hotels.html
/Hotels-g30976-Delta_Junction_Alaska-Hotels.html
/Hotels-g60833-Gustavus_Alaska-Hotels.html
/Hotels-g60998-Copper_Center_Alaska-Hotels.html
/Hotels-g31165-Wrangell_Alaska-Hotels.html
/Hotels-g31118-Seldovia_Alaska-Hotels.html
/Hotels-g11914881-McKinley_Village_Denali_National_Park_and_Preserve_Alaska-Hotels.html
/Hotels-g60877-Skagway_Alaska-Hotels.html
/Hotels-g30942-Bethel_Alaska-Hotels.html
/Hotels-g31079-North_Pole_Alaska-Hotels.html
/Hotels-g30997-Gakona_Alaska-Hotels.html
/Hotels-g6426461-Kachemak_Bay_State_Park_Alaska-Hotels.html
/Hotels-g31060-Moose_Pass_Alaska-Hotels.html
/Hotels-g30972-Cordova_Alaska-Hotels.html
/Hotels-g1655683-Glacier_View_Alaska-Hotels.html


In [ ]:
time.sleep(5)
hotels_toggle = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div/div[1]/div[1]/div[5]/div/div/div[2]/div[2]/div[2]/div[4]/div/label')
hotels_toggle.click()
soup = BeautifulSoup(driver.page_source)
while soup.find("div", id="component_14").find("div", class_="_16gKMTFp").find("div", class_="ui_pagination is-centered").find_all("a")[1].text == "Next":
    page_number += 1
    soup = BeautifulSoup(driver.page_source)
    reviews = soup.find_all("div", class_="_2wrUUKlw _3hFEdNs8")
    for review in reviews:
        reviewer, review_date = review.find("div", class_="_2fxQ4TOx").find("span").text.split(" wrote a review ")
        review_date = datetime.datetime.strptime(review_date.split(" ")[0], '%b').strftime('%B') + ' ' + review_date.split(" ")[1]
        review_link = f'https://www.tripadvisor.com{review.find("div", class_="glasR4aX").a["href"]}'
        review_title = review.find("div", class_="glasR4aX").a.span.text
        full_review = review.find("q", class_="IRsGHoPm").span.text
        if review.find("span", class_="_355y0nZn"):
            date_of_stay = review.find("span", class_="_34Xs-BQm").text.split(": ")[1]
        else:
            date_of_stay = np.NaN
        review_details_list = [property_name, property_address, property_lat_long, reviewer, review_date, date_of_stay, review_title, full_review, review_link]
        index = len(review_list_df)
        review_list_df.loc[index] = review_details_list
    with open("alaska_hotels.pickle", "wb") as to_write:
        pickle.dump(review_list_df, to_write)
    print(f'Page {page_number} done!')
    next_button = driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[7]/div/div[1]/div[1]/div/div/div[3]/div[8]/div/a[2]')
    next_button.click()
    soup = BeautifulSoup(driver.page_source)





driver = webdriver.Chrome(chromedriver)
driver.get("https://www.tripadvisor.com/Hotels-g28923-Alaska-Hotels.html")
time.sleep(5)
hotels_toggle = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div/div[1]/div[1]/div[5]/div/div/div[2]/div[2]/div[2]/div[4]/div/label')
hotels_toggle.click()
time.sleep(5)
soup = BeautifulSoup(driver.page_source)
urls = []
for i in range(98):
    soup = BeautifulSoup(driver.page_source)
    listings = soup.find("div", id="taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0").find_all("div", class_="prw_rup prw_meta_hsx_responsive_listing ui_section listItem")
    for listing in listings:
        urls.append(f'https://www.tripadvisor.com{listing.find("div", class_="listing_title").find("a")["href"]}')
    next_button = driver.find_element_by_xpath('//*[@id="taplc_main_pagination_bar_dusty_hotels_resp_0"]/div/div/div/span[2]')
    next_button.click()
soup = BeautifulSoup(driver.page_source)
listings = soup.find("div", id="taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0").find_all("div", class_="prw_rup prw_meta_hsx_responsive_listing ui_section listItem")
for listing in listings:
    urls.append(f'https://www.tripadvisor.com{listing.find("div", class_="listing_title").find("a")["href"]}')
urls

In [349]:
def get_alaska_hotel_urls():
    urls = []
    hotels = []

    driver = webdriver.Chrome(chromedriver)
    driver.get("https://www.tripadvisor.com/Hotels-g28923-Alaska-Hotels.html")
    hotels_toggle = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div/div[1]/div[1]/div[5]/div/div/div[2]/div[2]/div[2]/div[4]/div/label')
    hotels_toggle.click()
    for i in range(99):
        print(i)
        time.sleep(7)
        soup = BeautifulSoup(driver.page_source)
        listings = soup.find("div", id="taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0").find_all("div", class_="prw_rup prw_meta_hsx_responsive_listing ui_section listItem")
        for listing in listings:
            urls.append(f'https://www.tripadvisor.com{listing.find("div", class_="listing_title").find("a")["href"]}')
            hotels.append(listing.find("div", class_="listing_title").find("a").text)
        next_button = driver.find_element_by_xpath('//*[@id="taplc_main_pagination_bar_dusty_hotels_resp_0"]/div/div/div/span[2]')
        next_button.click()
        with open("hotel_urls.pickle", "wb") as to_write:
            pickle.dump(urls, to_write)

    return urls, hotels

In [350]:
hotel_urls, hotels = get_alaska_hotel_urls()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98


### For page 100

In [334]:
def get_last_page_alaska_hotel_urls():
    '''Scraping the last page of Alaska hotels (page 100) on Tripadvisor. Only necessary because I had the first 99 pages and my main function is not cooperating as I seek to finish the job (I bumped the range in the for loop up one to get the last page and the previously working part is not working).
    '''

    page_100_urls = []
    page_100_hotels = []

    driver = webdriver.Chrome(chromedriver)
    driver.get("https://www.tripadvisor.com/Hotels-g28923-Alaska-Hotels.html")
    time.sleep(7)
    hotels_toggle = driver.find_element_by_xpath('/html/body/div[2]/div[1]/div[2]/div/div[1]/div[1]/div[5]/div/div/div[2]/div[2]/div[2]/div[4]/div/label')
    hotels_toggle.click()
    time.sleep(7)
    next_button = driver.find_element_by_xpath('//*[@id="taplc_main_pagination_bar_dusty_hotels_resp_0"]/div/div/div/div/span[8]')
    next_button.click()
    time.sleep(7)
    soup = BeautifulSoup(driver.page_source)
    listings = soup.find("div", id="taplc_hsx_hotel_list_lite_dusty_hotels_combined_sponsored_0").find_all("div", class_="prw_rup prw_meta_hsx_responsive_listing ui_section listItem")
    for listing in listings:
        page_100_urls.append(f'https://www.tripadvisor.com{listing.find("div", class_="listing_title").find("a")["href"]}')
        page_100_hotels.append(listing.find("div", class_="listing_title").find("a").text)

    return page_100_urls, page_100_hotels

In [335]:
page_100_urls, page_100_hotels = get_last_page_alaska_hotel_urls()

In [368]:
with open("hotel_urls.pickle", "wb") as to_write:
    pickle.dump(hotel_urls, to_write)

In [453]:
with open("alaska_hotels_first_attempt.pickle", "wb") as to_write:
    pickle.dump(alaska_hotels_df_so_far, to_write)

In [632]:
get_reviews(hotel_urls[:325])

odasmom1148
TripGoat
TerraceFlorida
traveler15004
Page 293 done!
fotogenictravel
jes4317
DoloresSeattle
Grice-Fearing
BrightStar43
Page 294 done!
doneyForida
merf05
dino382
JacobLaCour
wjltwo
Page 295 done!
bjpremierontheroad
Mrsosci
debbiejo
DursleysFolks
jbtexas
Page 296 done!
texaskeystone
Washburns
Anon
FloridaAlaskabound
Sucheslife
Page 297 done!
mcoxpharmd
millersdad
pdxjeff1959
bgsualum
gabirds
Page 298 done!
Traveling Trish
Ariesgary
Jeff S
ccr770
Global12821
Page 299 done!
Ray R
travelstl
aviators99
CommercialBanker
EvanL
Page 300 done!
Page 301 (last) done!
 97%|█████████▋| 316/325 [14:23:46<1:15:48, 505.40s/it]Tiglax
Mark C
Mike B
Darien
jeezweez
Page 1 done!
Nina D
litg2017
CraigInTN
desertflower54
Donnalee C
Page 2 done!
_jmshb
Patti C
dayneanddanita
Chief400
Mary M
Page 3 done!
zarinafied
Amanda F
Alex M
cowgirlnurse1
Brett R
Page 4 done!
Sarah G
Austin C
lkowalski6
mvankley
IA_traveler
Page 5 done!
DrJG813
rainbowgirl1349
Brandon K
Turk M
ctina1166
Page 6 done!
Carol M
C

,Property name,Property address,"Lat, long",Reviewer,Review date,Date of stay,Review title,Full review,Review link
0,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",John M,October 2020,October 2020,Awesome hotel doing all the right things,I stay at lots of hotels (including during Cov...,https://www.tripadvisor.com/ShowUserReviews-g6...
1,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",Kenai Drift Anglers,October 2020,October 2020,Great Local Get Away,My girlfriend just wanted to get away from tow...,https://www.tripadvisor.com/ShowUserReviews-g6...
2,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",TravelGirl2006,October 2020,October 2020,We love this hotel,Have stayed here twice on our last two trips t...,https://www.tripadvisor.com/ShowUserReviews-g6...
3,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",maduro44,October 2020,September 2020,Bed,Everything was really nice almost great if it ...,https://www.tripadvisor.com/ShowUserReviews-g6...
4,Harbor 360 Hotel,"1412 4th Ave., Seward, AK 99664","60.119797,-149.439346",rtfields2020,October 2020,September 2020,Great place,"Terrific place to stay. Comfortable and clean,...",https://www.tripadvisor.com/ShowUserReviews-g6...
...,...,...,...,...,...,...,...,...,...
50455,Hatcher Pass Bed & Breakfast,"9000 N Palmer Fishhook Rd, Palmer, AK 99645-8043","61.686283,-149.241776",Lucyinsky,September 2007,September 2007,I Loved it Here!!,"My husband, parents, and I stayed here for two...",https://www.tripadvisor.com/ShowUserReviews-g3...
50456,Hatcher Pass Bed & Breakfast,"9000 N Palmer Fishhook Rd, Palmer, AK 99645-8043","61.686283,-149.241776",fsctb5,March 2007,March 2007,"Great Location, Very Cozy.","March, 2007 When we first pulled up to Hatcher...",https://www.tripadvisor.com/ShowUserReviews-g3...
50457,Hatcher Pass Bed & Breakfast,"9000 N Palmer Fishhook Rd, Palmer, AK 99645-8043","61.686283,-149.241776",LaughALot99,August 2006,August 2006,"Cozy and warming feeling, very clean",This was our second B&B. The first one in Sew...,https://www.tripadvisor.com/ShowUserReviews-g3...
50458,Hatcher Pass Bed & Breakfast,"9000 N Palmer Fishhook Rd, Palmer, AK 99645-8043","61.686283,-149.241776",meginoregon,May 2006,NaN,Super Nice,"We stayed at Hatcher Pass B&B in July 2005, an...",https://www.tripadvisor.com/ShowUserReviews-g3...


In [ ]:
get_reviews(hotel_urls[1000:])

# Skipped:
hotel_urls[138]
hotel_urls[165]
hotel_urls[188]
hotel_urls[228]
hotel_urls[324] (big one) (also started opening read more here)
hotel_urls[348]
hotle_urls[349]
350
351
352
431
453
488

In [629]:
hotel_urls[1589:1610]

['https://www.tripadvisor.com/VacationRentalReview-g31007-d19089880-Denali_Wild_Stay_MOOSE_CABIN-Healy_Denali_National_Park_and_Preserve_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g31091-d1628860-Reviews-Tyee_Lodge-Petersburg_Alaska.html',
 'https://www.tripadvisor.com/Hotel_Review-g29020-d17478749-Reviews-Tikaani_Lodge-Anchor_Point_Alaska.html',
 'https://www.tripadvisor.com/VacationRentalReview-g31127-d13988435-Arctic_Wolf_Lodge_W_Private_Dock_on_The_Kenai_River-Soldotna_Alaska.html',
 'https://www.tripadvisor.com/VacationRentalReview-g60880-d17443281-Rainy_Pass_Lodge_Main_Lodge-Anchorage_Alaska.html',
 'https://www.tripadvisor.com/VacationRentalReview-g31009-d10811635-Stunning_Panoramic_View_Upper_Lower_Whole_House-Homer_Alaska.html',
 'https://www.tripadvisor.com/VacationRentalReview-g2161460-d18894794-Beachside_Retreat_Oceanfront_home_on_main_channel-Kodiak_Kodiak_Island_Alaska.html',
 'https://www.tripadvisor.com/VacationRentalReview-g31020-d20816370-NEW_Adventure_R